In [ ]:
import json
from openai import AsyncOpenAI
from tqdm import tqdm

In [ ]:
BATCHSIZE = 64
SYSTEM_PROMPT = {'role':'system', 'content':'Vous êtes un assistant virtuel utile.'}
GPT_MODEL = "gpt-4o-2024-08-06"

In [ ]:
model_path = "models/4096_8_-1/"

In [ ]:
client = AsyncOpenAI()

In [ ]:
prompt_path = f"{model_path}/interpreter_prompts_v2.json"

In [ ]:
with open(prompt_path, "r") as f: 
    prompts = json.load(f)

prompts = [(int(k), v) for k, v in prompts.items()]

Below is inspired by guide at https://oshea00.github.io/posts/async-openai/ 

In [ ]:
async def async_completion(prompt):
    messages = [
        SYSTEM_PROMPT, 
        {'role':'user', 'content':f"{prompt}"}
    ]
    completion = await client.chat.completions.create(
        model=GPT_MODEL,
        messages=messages,
        temperature=0.2
    )
    return completion.choices[0].message.content

In [ ]:
async def reformat_completion_with_semaphore(key, prompt, semaphore): 
    async with semaphore: 
        response = await async_completion(prompt)
        return key, response

In [ ]:
import asyncio
from asyncio import Semaphore

async def run_batch(items): 
    semaphore = Semaphore(len(items))
    tasks = [reformat_completion_with_semaphore(key, prompt, semaphore)
             for key, prompt in items]
    results = await asyncio.gather(*tasks)
    return results

In [ ]:
async def get_completions(prompts): 
    completions = []
    for i in tqdm(range(0, len(prompts), 64)): 
            items = prompts[i:i+BATCHSIZE]
            results = await run_batch(items)
            completions += results
    return completions

In [ ]:
completions = await get_completions(prompts)

In [ ]:
completions = {k: v for k, v in completions}

with open(f"{model_path}/interpreter_responses_{GPT_MODEL}_v2.json", "w") as f: 
    json.dump(completions, f)